<a href="https://colab.research.google.com/github/Dinesh-Adhithya-H/Deep_Learning_using_Tensorflow/blob/main/C7_1_Transfer_Learning_InceptionV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip

--2021-05-30 17:22:05--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
Resolving download.microsoft.com (download.microsoft.com)... 184.84.168.111, 2600:1407:3c00:789::e59, 2600:1407:3c00:78f::e59
Connecting to download.microsoft.com (download.microsoft.com)|184.84.168.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824894548 (787M) [application/octet-stream]
Saving to: ‘kagglecatsanddogs_3367a.zip’

kagglecatsanddogs_3 100%[===================>] 786.68M   159MB/s    in 5.0s    

2021-05-30 17:22:10 (158 MB/s) - ‘kagglecatsanddogs_3367a.zip’ saved [824894548/824894548]



In [2]:
import zipfile
zip_ref=zipfile.ZipFile("/content/kagglecatsanddogs_3367a.zip","r")
zip_ref.extractall("/tmp/")
zip_ref.close()

In [3]:
import os
print(len(os.listdir("/tmp/PetImages/Cat")))
print(len(os.listdir("/tmp/PetImages/Dog")))

12501
12501


In [4]:
try:
  os.mkdir("/tmp/cats-v-dogs")
  os.mkdir("/tmp/cats-v-dogs/train")
  os.mkdir("/tmp/cats-v-dogs/test")
  os.mkdir("/tmp/cats-v-dogs/train/dogs")
  os.mkdir("/tmp/cats-v-dogs/train/cats")
  os.mkdir("/tmp/cats-v-dogs/test/dogs")
  os.mkdir("/tmp/cats-v-dogs/test/cats")
except OSError:
  pass

In [5]:
from shutil import copyfile
for i in os.listdir("/tmp/PetImages/Cat")[:int(12501*0.8)]:
  path="/tmp/PetImages/Cat/"+i
  if os.path.getsize(path)>0:
    copyfile(path,"/tmp/cats-v-dogs/train/cats/"+i)
for i in os.listdir("/tmp/PetImages/Cat")[int(12501*0.8):]:
  path="/tmp/PetImages/Cat/"+i
  if os.path.getsize(path)>0:
    copyfile(path,"/tmp/cats-v-dogs/test/cats/"+i)

In [6]:
for i in os.listdir("/tmp/PetImages/Dog")[:int(12501*0.8)]:
  path="/tmp/PetImages/Dog/"+i
  if os.path.getsize(path)>0:
    copyfile(path,"/tmp/cats-v-dogs/train/dogs/"+i)
for i in os.listdir("/tmp/PetImages/Dog")[int(12501*0.8):]:
  path="/tmp/PetImages/Dog/"+i
  if os.path.getsize(path)>0:
    copyfile(path,"/tmp/cats-v-dogs/test/dogs/"+i)

In [9]:
len(os.listdir("/tmp/cats-v-dogs/train/dogs")),len(os.listdir("/tmp/cats-v-dogs/train/cats")),len(os.listdir("/tmp/cats-v-dogs/test/dogs")),len(os.listdir("/tmp/cats-v-dogs/test/cats"))

(9999, 9999, 2501, 2501)

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1/255.0,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest")
train_generator=train_datagen.flow_from_directory("/tmp/cats-v-dogs/train/",
                batch_size=100,
                class_mode="binary",
                target_size=(150,150))

Found 19996 images belonging to 2 classes.


In [12]:
test_datagen=ImageDataGenerator(rescale=1/255.0)
test_generator=test_datagen.flow_from_directory("/tmp/cats-v-dogs/test/",
          batch_size=100,
          class_mode="binary",
          target_size=(150,150))

Found 5002 images belonging to 2 classes.


In [16]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
weights_url = "https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
weights_file="inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
import urllib.request
urllib.request.urlretrieve(weights_url, weights_file)
pre_trained_model=InceptionV3(input_shape=(150,150,3),
        include_top=False,
        weights=None)
pre_trained_model.load_weights(weights_file)
for i in pre_trained_model.layers:
  i.trainable=False
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d_188 (Conv2D)             (None, 74, 74, 32)   864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_188 (BatchN (None, 74, 74, 32)   96          conv2d_188[0][0]                 
__________________________________________________________________________________________________
activation_188 (Activation)     (None, 74, 74, 32)   0           batch_normalization_188[0][0]    
_______________________________________________________________________________________

In [20]:
last_layer=pre_trained_model.get_layer("mixed10")
last_output=last_layer.output

In [23]:
last_layer

In [21]:
from tensorflow.keras import layers,Model
x=layers.Flatten()(last_output)
x=layers.Dense(512,activation="relu")(x)
x=layers.Dense(1,activation="sigmoid")(x)
model=Model(pre_trained_model.input,x)

In [24]:
model.compile(optimizer="RMSprop",loss="binary_crossentropy",metrics=["acc"])

In [26]:
history=model.fit(train_generator,epochs=1,validation_data=test_generator,verbose=1)

200/200 [==============================] - ETA: 0s - loss: 0.3098 - acc: 0.8882

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only

200/200 [==============================] - 1108s 6s/step - loss: 0.3098 - acc: 0.8882 - val_loss: 0.0900 - val_acc: 0.9666


In [27]:
acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

In [28]:
acc,val_acc,loss,val_loss

([0.8881776332855225],
 [0.9666133522987366],
 [0.30980122089385986],
 [0.08999006450176239])